In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#import dataset
dataset = pd.read_csv("TSLA.csv")

In [ ]:
dataset.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2956.000000,2956.000000,2956.000000,2956.000000,2956.000000,2.956000e+03
mean,138.691296,141.771603,135.425953,138.762183,138.762183,3.131449e+07
std,250.044839,255.863239,243.774157,250.123115,250.123115,2.798383e+07
min,3.228000,3.326000,2.996000,3.160000,3.160000,5.925000e+05
25%,19.627000,20.402000,19.127500,19.615000,19.615000,1.310288e+07
50%,46.656999,47.487001,45.820002,46.545000,46.545000,2.488680e+07
75%,68.057001,69.357500,66.911501,68.103998,68.103998,3.973875e+07
max,1234.410034,1243.489990,1217.000000,1229.910034,1229.910034,3.046940e+08


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2956 entries, 0 to 2955
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2956 non-null   object 
 1   Open       2956 non-null   float64
 2   High       2956 non-null   float64
 3   Low        2956 non-null   float64
 4   Close      2956 non-null   float64
 5   Adj Close  2956 non-null   float64
 6   Volume     2956 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 161.8+ KB


In [ ]:
dataset.head()

NameError: name 'dataset' is not defined

In [ ]:
#scale values to range 0-1
trainset = dataset.iloc[:, 3:4].values
scaler = MinMaxScaler(feature_range = (0,1))
data_scaled = scaler.fit_transform(trainset)
data_scaled

array([[4.21744904e-04],
       [1.37067094e-03],
       [8.71496305e-04],
       ...,
       [7.56796518e-01],
       [8.01812864e-01],
       [8.12026969e-01]])

In [ ]:
# split dataset to train and test sets
x_train, y_train = [], []
x_test, y_test = [], []

window = 70

# Training set
for i in range(window, len(data_scaled)):
    x_train.append(data_scaled[i - window:i, 0])
    y_train.append(data_scaled[i, 0])

# Testing set
for i in range(len(data_scaled) - window, len(data_scaled)):
    x_test.append(data_scaled[i - window:i, 0])
    y_test.append(data_scaled[i, 0])

# Convert lists to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))

NameError: name 'x_train' is not defined

In [ ]:
#build LSTM neural network model
regressor = Sequential()
regressor.add(LSTM(units = 50,return_sequences = True,input_shape = (x_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam',loss = 'mean_squared_error')

NameError: name 'x_train' is not defined

In [ ]:
#train model
regressor.fit(x_train, y_train,epochs = 30, batch_size = 32)

Epoch 1/30
91/91 [==============================] - 15s 74ms/step - loss: 0.0054
Epoch 2/30
91/91 [==============================] - 7s 79ms/step - loss: 0.0016
Epoch 3/30
91/91 [==============================] - 7s 83ms/step - loss: 0.0015
Epoch 4/30
91/91 [==============================] - 7s 77ms/step - loss: 0.0011
Epoch 5/30
91/91 [==============================] - 7s 80ms/step - loss: 0.0013
Epoch 6/30
91/91 [==============================] - 7s 81ms/step - loss: 0.0012
Epoch 7/30
91/91 [==============================] - 7s 77ms/step - loss: 0.0014
Epoch 8/30
91/91 [==============================] - 7s 81ms/step - loss: 0.0013
Epoch 9/30
91/91 [==============================] - 7s 74ms/step - loss: 0.0011
Epoch 10/30
91/91 [==============================] - 7s 81ms/step - loss: 0.0010
Epoch 11/30
91/91 [==============================] - 7s 80ms/step - loss: 8.7482e-04
Epoch 12/30
91/91 [==============================] - 7s 75ms/step - loss: 0.0010
Epoch 13/30
91/91 [=============

In [ ]:
#generate prices for test data
predicted_price = regressor.predict(x_test)
predicted_price = scaler.inverse_transform(predicted_price)
predicted_price

3/3 [==============================] - 1s 23ms/step


array([[ 946.9541 ],
       [ 938.5707 ],
       [ 926.66797],
       [ 914.16956],
       [ 902.6323 ],
       [ 892.164  ],
       [ 882.92145],
       [ 881.91077],
       [ 893.50806],
       [ 921.2453 ],
       [ 958.0458 ],
       [ 990.36316],
       [1008.7183 ],
       [1012.71686],
       [1015.28467],
       [1019.34937],
       [1019.90326],
       [1010.1387 ],
       [ 992.2273 ],
       [ 969.70044],
       [ 953.804  ],
       [ 952.101  ],
       [ 958.73944],
       [ 966.23865],
       [ 971.0553 ],
       [ 970.5281 ],
       [ 965.7261 ],
       [ 954.1647 ],
       [ 930.83545],
       [ 905.1631 ],
       [ 884.5784 ],
       [ 866.1088 ],
       [ 846.6057 ],
       [ 832.94446],
       [ 831.72394],
       [ 840.8146 ],
       [ 853.72614],
       [ 864.91974],
       [ 873.5249 ],
       [ 878.3875 ],
       [ 881.87555],
       [ 882.6377 ],
       [ 876.542  ],
       [ 865.1204 ],
       [ 855.4954 ],
       [ 851.7375 ],
       [ 851.4681 ],
       [ 849.

In [ ]:
#get real price for comparison
real_price =  scaler.inverse_transform(y_test.reshape(-1, 1))
real_price

NameError: name 'scaler' is not defined

In [ ]:
#plot predicted and real price
plt.plot(real_price, color = 'orange', label = 'real price')
plt.plot(predicted_price, color = 'green', label = 'predicted price')
plt.title('Google stock price prediction')
plt.xlabel('time')
plt.ylabel('stock price')
plt.legend()
plt.show()

NameError: name 'real_price' is not defined

In [ ]:
#metrics
print(f'RMSE {metrics.mean_squared_error(predicted_price, real_price, squared=False)}')
print(f'MAE {metrics.mean_absolute_error(predicted_price, real_price)}')

RMSE 78.24165488295493
MAE 60.62151321333704
